Connected to py39 (Python 3.9.19)

In [5]:
import torch
import numpy as np

In [6]:
import sys
import os

# Add the current file's directory to the Python path
current_dir = os.path.dirname(os.path.abspath('..'))
if current_dir not in sys.path:
    sys.path.append(current_dir)

from solution import *

In [ ]:
# Construct the full range of uint8 for e4m3
x_e4m3 = torch.arange(0, 2**8, dtype=torch.int32)
s_e4m3, e_e4m3, m_e4m3 = decompose_8bit_e4m3(x_e4m3)

# Generate fp8 e4m3 as floating points
fp8_e4m3 = torch.zeros(256, dtype=torch.float32)

for i in range(256):
    sign = (i & 0b10000000) >> 7
    exponent = (i & 0b01111000) >> 3
    mantissa = i & 0b00000111

    # Handle special cases
    if exponent == 0:
        if mantissa == 0:
            # Zero
            fp8_e4m3[i] = 0.0 if sign == 0 else -0.0
        else:
            # Subnormal numbers
            fp8_e4m3[i] = ((-1)**sign) * (2**-6) * (mantissa / 8)
    elif exponent == 15:
        if mantissa == 0:
            # Infinity
            fp8_e4m3[i] = float('inf') if sign == 0 else float('-inf')
        else:
            # NaN
            fp8_e4m3[i] = float('nan')
    else:
        # Normal numbers
        fp8_e4m3[i] = ((-1)**sign) * (2**(exponent - 7)) * (1 + mantissa / 8)

print("FP8 E4M3 range:")
print(fp8_e4m3)


FP8 E4M3 range:
tensor([ 0.0000e+00,  1.9531e-03,  3.9062e-03,  5.8594e-03,  7.8125e-03,
         9.7656e-03,  1.1719e-02,  1.3672e-02,  1.5625e-02,  1.7578e-02,
         1.9531e-02,  2.1484e-02,  2.3438e-02,  2.5391e-02,  2.7344e-02,
         2.9297e-02,  3.1250e-02,  3.5156e-02,  3.9062e-02,  4.2969e-02,
         4.6875e-02,  5.0781e-02,  5.4688e-02,  5.8594e-02,  6.2500e-02,
         7.0312e-02,  7.8125e-02,  8.5938e-02,  9.3750e-02,  1.0156e-01,
         1.0938e-01,  1.1719e-01,  1.2500e-01,  1.4062e-01,  1.5625e-01,
         1.7188e-01,  1.8750e-01,  2.0312e-01,  2.1875e-01,  2.3438e-01,
         2.5000e-01,  2.8125e-01,  3.1250e-01,  3.4375e-01,  3.7500e-01,
         4.0625e-01,  4.3750e-01,  4.6875e-01,  5.0000e-01,  5.6250e-01,
         6.2500e-01,  6.8750e-01,  7.5000e-01,  8.1250e-01,  8.7500e-01,
         9.3750e-01,  1.0000e+00,  1.1250e+00,  1.2500e+00,  1.3750e+00,
         1.5000e+00,  1.6250e+00,  1.7500e+00,  1.8750e+00,  2.0000e+00,
         2.2500e+00,  2.5000e+00,  

In [ ]:
# Construct the full range of uint8
x = torch.arange(0, 2**8, dtype=torch.int32)
s, e, m = decompose_8bit_e5m2(x)
# Generate fp8 e5m2 as floating points
fp8_e5m2 = torch.zeros(256, dtype=torch.float32)

for i in range(256):
    sign = (i & 0b10000000) >> 7
    exponent = (i & 0b01111100) >> 2
    mantissa = i & 0b00000011

    # Handle special cases
    if exponent == 0:
        if mantissa == 0:
            # Zero
            fp8_e5m2[i] = 0.0 if sign == 0 else -0.0
        else:
            # Subnormal numbers
            fp8_e5m2[i] = ((-1)**sign) * (2**-14) * (mantissa / 4)
    elif exponent == 31:
        if mantissa == 0:
            # Infinity
            fp8_e5m2[i] = float('inf') if sign == 0 else float('-inf')
        else:
            # NaN
            fp8_e5m2[i] = float('nan')
    else:
        # Normal numbers
        fp8_e5m2[i] = ((-1)**sign) * (2**(exponent - 15)) * (1 + mantissa / 4)

print("FP8 E5M2 range:")
print(fp8_e5m2)


FP8 E5M2 range:
tensor([ 0.0000e+00,  1.5259e-05,  3.0518e-05,  4.5776e-05,  6.1035e-05,
         7.6294e-05,  9.1553e-05,  1.0681e-04,  1.2207e-04,  1.5259e-04,
         1.8311e-04,  2.1362e-04,  2.4414e-04,  3.0518e-04,  3.6621e-04,
         4.2725e-04,  4.8828e-04,  6.1035e-04,  7.3242e-04,  8.5449e-04,
         9.7656e-04,  1.2207e-03,  1.4648e-03,  1.7090e-03,  1.9531e-03,
         2.4414e-03,  2.9297e-03,  3.4180e-03,  3.9062e-03,  4.8828e-03,
         5.8594e-03,  6.8359e-03,  7.8125e-03,  9.7656e-03,  1.1719e-02,
         1.3672e-02,  1.5625e-02,  1.9531e-02,  2.3438e-02,  2.7344e-02,
         3.1250e-02,  3.9062e-02,  4.6875e-02,  5.4688e-02,  6.2500e-02,
         7.8125e-02,  9.3750e-02,  1.0938e-01,  1.2500e-01,  1.5625e-01,
         1.8750e-01,  2.1875e-01,  2.5000e-01,  3.1250e-01,  3.7500e-01,
         4.3750e-01,  5.0000e-01,  6.2500e-01,  7.5000e-01,  8.7500e-01,
         1.0000e+00,  1.2500e+00,  1.5000e+00,  1.7500e+00,  2.0000e+00,
         2.5000e+00,  3.0000e+00,  

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm

def compute_inner_product_fp8(x, y, n_mantissa):
    x_tensor = torch.tensor(x, dtype=torch.bfloat16)
    y_tensor = torch.tensor(y, dtype=torch.bfloat16)
    
    x_fp8 = round_to_fp8_represented_as_int8(x_tensor, n_mantissa, None)
    y_fp8 = round_to_fp8_represented_as_int8(y_tensor, n_mantissa, None)
    
    x_fp8_float = undo_int8_fp8(x_fp8, n_mantissa)
    y_fp8_float = undo_int8_fp8(y_fp8, n_mantissa)
    
    return torch.dot(x_fp8_float.flatten(), y_fp8_float.flatten()).item()

def backward_error(true_result, computed_result):
    return np.abs(true_result - computed_result) / np.abs(true_result)

# Generate n values
n_values = np.logspace(2, 5, num=19, dtype=int)

# Initialize lists to store results
backward_errors_e5m2 = []
backward_errors_e4m3 = []

# Compute inner products and backward errors
for n in tqdm(n_values, desc="Computing errors"):
    # Generate x and y vectors at bfloat16 precision
    x = torch.rand(n, dtype=torch.bfloat16).to(torch.float32).numpy()
    y = torch.rand(n, dtype=torch.bfloat16).to(torch.float32).numpy()

    
    # Compute true result in double precision
    true_result = np.dot(x, y)
    
    # Compute result in e5m2 precision
    e5m2_result = compute_inner_product_fp8(x, y, 2)
    
    # Compute result in e4m3 precision
    e4m3_result = compute_inner_product_fp8(x, y, 3)
    
    # Compute backward errors
    error_e5m2 = backward_error(true_result, e5m2_result)
    error_e4m3 = backward_error(true_result, e4m3_result)
    
    backward_errors_e5m2.append(error_e5m2)
    backward_errors_e4m3.append(error_e4m3)

# Create the plot
plt.figure(figsize=(10, 6))
plt.loglog(n_values, backward_errors_e5m2, 'bo-', label='E5M2')
plt.loglog(n_values, backward_errors_e4m3, 'ro-', label='E4M3')

# Add sqrt(nu) line
eps = 0.0625  # e5m2 epsilon (2^-4)
sqrt_nu = np.sqrt(n_values) * eps
plt.loglog(n_values, sqrt_nu, 'g--', label='sqrt(nu)')

plt.xlabel('Vector length (n)')
plt.ylabel('Backward error')
plt.title('Backward Error of Inner Product in FP8 Precision')
plt.legend()
plt.grid(True)
plt.show()

print("E5M2 results:")
print(f"Minimum error: {min(backward_errors_e5m2):.2e}")
print(f"Maximum error: {max(backward_errors_e5m2):.2e}")
print(f"Mean error: {np.mean(backward_errors_e5m2):.2e}")

print("\nE4M3 results:")
print(f"Minimum error: {min(backward_errors_e4m3):.2e}")
print(f"Maximum error: {max(backward_errors_e4m3):.2e}")
print(f"Mean error: {np.mean(backward_errors_e4m3):.2e}")


ModuleNotFoundError: No module named 'matplotlib'

At this point we are able to:
- Construct the full range of bfloat16, uint8
- Convert bfloat16 to bits and back
- Convert bfloat16 to sign, exponent, mantissa and back
- Test that conversion is correct except NaN values
- Truncate when bfloat16 to e5m2 with mantissa chopping
- Round up when converting from bfloat16 to e5m2
- Round by stochastic rounding scheme

Next:
- Test using subnormal bfloat16 inputs
- Test clamping to e5m2 and e4m3